## Module 4 Final Project Submission - Computer Vision

* Student name: Emily J. Cain
* Student pace: Full Time
* Scheduled project review date/time: 
* Instructor name: Rafael Carrasco
* Blog post URL: https://medium.com/@magicmilly


#### Here I will write about my awesome project and why it's important

## I. Load & Organize Data
## II. Inspection & Exploratory Data Analysis
## III. Preprocessing
## IV. Modeling 
## V. Model Evaluation
## VI. Conclusions & Recommendations

In [ ]:
import cv2
import datetime
import numpy as np
import os
import pandas as pd
import random
import scipy
# import shutil
# import time
import warnings
# import zipfile

from keras import layers
from keras import models 
# from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.utils import to_categorical 
from matplotlib.pyplot import imshow
from PIL import Image
# from scipy import ndimage
# from skimage.io import imread 
# from skimage.transform import resize

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
warnings.filterwarnings(action='ignore', category=FutureWarning)

# start = datetime.datetime.now()

## I. Load & Organize Data

In [ ]:
# calculate amount of images
cnv_imgs = len(os.listdir('Data/cnv'))
dme_imgs = len(os.listdir('Data/dme'))
drusen_imgs = len(os.listdir('Data/drusen'))
normal_imgs = len(os.listdir('Data/normal'))

total_imgs = cnv_imgs + dme_imgs + drusen_imgs + normal_imgs
print(f'There are {total_imgs} total images.')

In [ ]:
# inspect an original image
test_img = Image.open('Data/cnv/CNV-6294785-1.jpeg')
# test_img.show() - this opens image in new window
imshow(np.asarray(test_img)); # imshow for inline image

#### Resize and label images

In [ ]:
img_classes = ['normal', 'cnv', 'dme', 'drusen']
img_size = 40
original_dir = 'Data'

if data:
    data = data
else:
    data = []

if not data:
    for img_class in img_classes:

        path = os.path.join(original_dir, img_class)
        # assign numeric value to class labels
        class_num = img_classes.index(img_class)

        for img in os.listdir(path):

            file = os.path.join(path, img)
            if file.endswith('.jpeg'):
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                # resize images to (40,40)
                resize_array = cv2.resize(img_array, (img_size, img_size))
                # create new data list of resized images with their class labels
                data.append([resize_array, class_num])
            else:
                print(file)
else:
    print('Data dun loaded, dood')
    print(len(data))

In [ ]:
np.save('./Data/img_array', data)

In [ ]:
db = np.load("./Data/img_array.npy")

In [ ]:
db.shape

In [ ]:
data = np.array(data)

In [ ]:
data.shape

In [ ]:
db[0][1]

In [ ]:
data[0]

verify